In [18]:
from typing import Annotated
from dotenv import load_dotenv
load_dotenv()

from typing_extensions import TypedDict
from langchain.chat_models import init_chat_model
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages

In [19]:
from langchain.chat_models import init_chat_model
llm = init_chat_model("google_genai:gemini-2.5-flash-lite")
class State(TypedDict):
    messages: Annotated[list, add_messages]

def chatbot(state: State) -> State:
    return {"messages": [llm.invoke(state["messages"])]}

builder = StateGraph(State)
builder.add_node("chatbot_node", chatbot)

builder.add_edge(START, "chatbot_node")
builder.add_edge("chatbot_node", END)

graph = builder.compile()

In [20]:
message = {"role": "user", "content": "Who walked on the moon for the first time? Print only the name"}
# message = {"role": "user", "content": "What is the latest price of MSFT stock?"}
response = graph.invoke({"messages":[message]})

response["messages"]

[HumanMessage(content='Who walked on the moon for the first time? Print only the name', additional_kwargs={}, response_metadata={}, id='0429fe3f-d8f1-468e-acb6-913ce1b4cc61'),
 AIMessage(content='Neil Armstrong', additional_kwargs={}, response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash-lite', 'safety_ratings': [], 'model_provider': 'google_genai'}, id='lc_run--019b9178-d5f3-7bf2-9fee-cf3aab6b9d9b-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 15, 'output_tokens': 2, 'total_tokens': 17, 'input_token_details': {'cache_read': 0}})]

In [21]:
state = None
while True:
    in_message = input("You: ")
    if in_message.lower() in {"quit","exit"}:
        break
    if state is None:
        state: State = {
            "messages": [{"role": "user", "content": in_message}]
        }
    else:
        state["messages"].append({"role": "user", "content": in_message})

    state = graph.invoke(state)
    print("Bot:", state["messages"][-1].content)

Bot: Hey there! How can I help you today? 😊
Bot: Haha, okay! What's "that"? Are you feeling a certain way, or did something happen that you're referring to?

Give me a little more context, and I'll do my best to understand! 😄
Bot: "Done" with what?

Are you:

*   **Done with a task?** Like, "I'm done with my homework."
*   **Done with something you don't like?** Like, "I'm done with this boring movie."
*   **Done with the conversation?** Like, "I'm done talking about this."
*   **Done with a cooking recipe?** Like, "The cake is done!"

Let me know what you mean by "done"! I'm curious! 🤔
Bot: Okay, I understand. If you're done talking to me, I'll stop responding.

It was nice chatting with you! Have a good day!
Bot: I can't directly control your browser or how you're interacting with me. I'm a text-based AI, and the "dialogue box" you're seeing is likely part of the interface of the application or website you're using to talk to me.

To stop seeing this, you'll need to close the tab, wi

KeyboardInterrupt: Interrupted by user